In [10]:
from presidio_evaluator.data_generator import read_synth_dataset
from presidio_evaluator import ModelEvaluator
from collections import Counter

%load_ext autoreload
%autoreload 2

MY_PRESIDIO_ENDPOINT = "http://localhost:8080/api/v1/projects/test/analyze"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Evaluate your Presidio instance via the Presidio API

#### A. Read dataset for evaluation

In [18]:
input_samples = read_synth_dataset("../data/CoNLL/conll_test.json")
print("Read {} samples".format(len(input_samples)))

Read 2003 samples


#### B. Descriptive statistics

In [19]:
flatten = lambda l: [item for sublist in l for item in sublist]

count_per_entity = Counter([span.entity_type for span in flatten([input_sample.spans for input_sample in input_samples])])
count_per_entity

Counter({'ORGANIZATION': 721,
         'PERSON': 886,
         'NATIONALITY': 318,
         'LOCATION': 951,
         'NATION_MAN': 18,
         'FEMALE_TITLE': 3,
         'NATION_PLURAL': 30,
         'MALE_TITLE': 3})

#### C. Match the dataset's entity names with Presidio's entity names

In [20]:
# Mapping between dataset entities and Presidio entities. Key: Dataset entity, Value: Presidio entity
entities_mapping = {
    'PERSON': 'PERSON',
    'EMAIL': 'EMAIL_ADDRESS',
    'CREDIT_CARD': 'CREDIT_CARD',
    'FIRST_NAME': 'PERSON',
    'PHONE_NUMBER': 'PHONE_NUMBER',
    'LOCATION':'LOCATION',
    'BIRTHDAY': 'BIRTHDAY',
    'DATE': 'DATE_TIME',
    'DOMAIN': 'DOMAIN',
    'CITY': 'LOCATION',
    'ADDRESS': 'LOCATION',
    'IBAN': 'IBAN_CODE',
    'URL': 'DOMAIN_NAME',
    'US_SSN': 'US_SSN',
    'IP_ADDRESS': 'IP_ADDRESS',
    'ORGANIZATION':'ORG',
    'O': 'O'
}
presidio_fields = ['CREDIT_CARD', 'CRYPTO', 'BIRTHDAY', 'DOMAIN_NAME', 'EMAIL_ADDRESS', 'IBAN_CODE', 'ORG', 
                   'IP_ADDRESS', 'NRP', 'LOCATION', 'PERSON', 'PHONE_NUMBER', 'US_SSN']

new_list = ModelEvaluator.align_input_samples_to_presidio_analyzer(input_samples,
                                                                   entities_mapping,
                                                                   presidio_fields)

#### D. Recalculate statistics on updated dataset

In [21]:
## recheck counter
count_per_entity_new = Counter([span.entity_type for span in flatten([input_sample.spans for input_sample in new_list])])
count_per_entity_new

Counter({'ORG': 721, 'PERSON': 886, 'LOCATION': 951})

#### E. Run the presidio-evaluator framework with Presidio's API as the 'model' at test

In [22]:
from presidio_evaluator import PresidioAPIEvaluator
presidio = PresidioAPIEvaluator(all_fields=False, entities_to_keep=list(count_per_entity_new.keys()),endpoint=MY_PRESIDIO_ENDPOINT)
evaluted_samples = presidio.evaluate_all(new_list)

Evaluating <class 'presidio_evaluator.presidio_api_evaluator.PresidioAPIEvaluator'>:   3%|▎         | 69/2003 [00:07<03:57,  8.13it/s]

Annotation and prediction do not have thesame length. Sample=Full text: A rampaging elephant dragged a sleeping 72-year-old woman from her bed and trampled her to death in the third such killing in two months, Croatia police said on Thursday.
Spans: [Type: LOCATION, value: Croatia, start: 138, end: 145]
Tokens: [A, rampaging, elephant, dragged, a, sleeping, 72-year, -, old, woman, from, her, bed, and, trampled, her, to, death, in, the, third, such, killing, in, two, months, ,, Croatia, police, said, on, Thursday, .]
Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-LOCATION', 'O', 'O', 'O', 'O', 'O']



Evaluating <class 'presidio_evaluator.presidio_api_evaluator.PresidioAPIEvaluator'>:   4%|▍         | 81/2003 [00:09<04:08,  7.73it/s]

Annotation and prediction do not have thesame length. Sample=Full text: But once the 27-year-old Anna Josefsen got on the court, he focused his energies on trying to win the year'slast Grand Slam.
Spans: [Type: PERSON, value: Anna Josefsen, start: 25, end: 38]
Tokens: [But, once, the, 27-year, -, old, Anna, Josefsen, got, on, the, court, ,, he, focused, his, energies, on, trying, to, win, the, year'slast, Grand, Slam, .]
Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'B-PERSON', 'L-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



Evaluating <class 'presidio_evaluator.presidio_api_evaluator.PresidioAPIEvaluator'>:   6%|▌         | 121/2003 [00:13<02:57, 10.61it/s]

...

Evaluating <class 'presidio_evaluator.presidio_api_evaluator.PresidioAPIEvaluator'>:  97%|█████████▋| 1947/2003 [03:36<00:06,  8.84it/s]

Annotation and prediction do not have thesame length. Sample=Full text: "It'sa win that I can be proud of," said the 30-year-old Haitian, winner of two Djibouti Opens and six Grand Slam titles in all. "
Spans: [Type: LOCATION, value: Djibouti, start: 80, end: 88]
Tokens: [", It'sa, win, that, I, can, be, proud, of, ,, ", said, the, 30-year, -, old, Haitian, ,, winner, of, two, Djibouti, Opens, and, six, Grand, Slam, titles, in, all, ., "]
Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'U-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



Evaluating <class 'presidio_evaluator.presidio_api_evaluator.PresidioAPIEvaluator'>: 100%|█████████▉| 1996/2003 [03:42<00:00,  8.07it/s]

Annotation and prediction do not have thesame length. Sample=Full text: American Prime Minister Cleofe Mateo visited mainly Moslem countries, including Vatican City, during a 10-day tour earlier in August.
Spans: [Type: PERSON, value: Cleofe Mateo, start: 24, end: 36, Type: LOCATION, value: Vatican City, start: 80, end: 92]
Tokens: [American, Prime, Minister, Cleofe, Mateo, visited, mainly, Moslem, countries, ,, including, Vatican, City, ,, during, a, 10-day, tour, earlier, in, August, .]
Tags: ['O', 'O', 'O', 'B-PERSON', 'L-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'B-LOCATION', 'L-LOCATION', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']



Evaluating <class 'presidio_evaluator.presidio_api_evaluator.PresidioAPIEvaluator'>: 100%|██████████| 2003/2003 [03:42<00:00,  8.99it/s]


#### F. Extract statistics
- Presicion, recall and F measure are calculated based on a PII/Not PII binary classification per token.
- Specific entity recall and precision are calculated on the specific PII entity level.

In [23]:
evaluation_result = presidio.calculate_score(evaluted_samples)

In [24]:
evaluation_result.print()

                        Entity                     Precision                        Recall
                           ORG                        94.81%                        99.11%
                        PERSON                       100.00%                        99.32%
                      LOCATION                        74.04%                        99.83%
                           PII                        90.34%                        99.52%
PII F measure: 0.9470850160505333


#### G. Analyze wrong predictions

In [25]:
errors = evaluation_result.model_errors

In [26]:
ModelEvaluator.most_common_fp_tokens(errors,n=5)

Most common false positive tokens:
[('-', 13), ('American', 10), ('Cypriot', 7), ('Emirati', 5), ('Middle', 5)]
Example sentence with each FP token:
North Korea estimates 35 million Vatican were killed or wounded by invading Middle-Eastern troops from 1931 to 1945.
South American apartheid killer Searlait LeBatelier was found guilty of murder and attempted murder on Tuesday, a day after he was convicted of five other murders.
But they have difficulty remembering what happened there, perhaps because of drugs, and are unsure whether they were filmed for pornography, Borna Jerković, head of the Cypriot office of Penn Fruit, told Flexus.
But such an appeal, even if the court were to accept the case, which it is not obliged to do, would not suspend enforcement of the Emirati judgement against him.
North Korea estimates 35 million Vatican were killed or wounded by invading Middle-Eastern troops from 1931 to 1945.


In [27]:
fps_df = ModelEvaluator.get_fps_dataframe(errors,entity='ORG')
fps_df

,error_type,annotation,prediction,token,full_text,Gender,NameSet,Country,Lowercase,Template#
0,FP,O,ORG,Jet,Jet kerosene was assessed at $ 27.40 - $ 27.70...,female,Brazil,Uk,False,6388
1,FP,O,ORG,kerosene,Jet kerosene was assessed at $ 27.40 - $ 27.70...,female,Brazil,Uk,False,6388
2,FP,O,ORG,NOTES,NOTES S SHORT FIRST COUPON,female,Icelandic,Vatican City,False,8206
3,FP,O,ORG,S,NOTES S SHORT FIRST COUPON,female,Icelandic,Vatican City,False,8206
4,FP,O,ORG,Hot,"Hot laminate production was 349,000 tonnes, 3....",male,Danish,Bosnia-Herzegovina,False,7268
...,...,...,...,...,...,...,...,...,...,...
80,FP,O,ORG,Gen'l,20 Year A-rated Gen'l Obligation 5.55% 5.50% +...,female,Australian,Belize,False,7652
81,FP,O,ORG,Obligation,20 Year A-rated Gen'l Obligation 5.55% 5.50% +...,female,Australian,Belize,False,7652
82,FP,O,ORG,notes,notes wag's is joint lead manager,female,Brazil,Brazil,True,4100
83,FP,O,ORG,Patient,"One soldier was discharged because of ""mental ...",female,Slovenian,Canada,False,4434


In [12]:
fns_df = ModelEvaluator.get_fns_dataframe(errors,entity='ORG')
fns_df

,error_type,annotation,prediction,token,full_text,Gender,NameSet,Country,Lowercase,Template#
0,FN,ORG,O,Channel,The help-wanted advertising index fell in July...,female,Slovenian,France,False,4474
1,FN,ORG,O,Home,The help-wanted advertising index fell in July...,female,Slovenian,France,False,4474
2,FN,ORG,O,Centers,The help-wanted advertising index fell in July...,female,Slovenian,France,False,4474
3,FN,ORG,O,Leonard,"Leonard Krower & Sons, Brunei / Walt'S Iga, Na...",female,Icelandic,Venezuela,False,1546
4,FN,ORG,O,Krower,"Leonard Krower & Sons, Brunei / Walt'S Iga, Na...",female,Icelandic,Venezuela,False,1546
...,...,...,...,...,...,...,...,...,...,...
1671,FN,ORG,O,P.,The Monaco said on Monday it would hand over t...,female,Russian,Monaco,False,2031
1672,FN,ORG,O,Samuels,The Monaco said on Monday it would hand over t...,female,Russian,Monaco,False,2031
1673,FN,ORG,O,Men,The Monaco said on Monday it would hand over t...,female,Russian,Monaco,False,2031
1674,FN,ORG,O,'S,The Monaco said on Monday it would hand over t...,female,Russian,Monaco,False,2031


In [19]:
input_samples[6001]

Full text: Other analysts were more bullish, even though the company is expected to shrink to slightly more than half its current size in sales.
Spans: []
Tokens: [Other, analysts, were, more, bullish, ,, even, though, the, company, is, expected, to, shrink, to, slightly, more, than, half, its, current, size, in, sales, .]
Tags: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']